## Compare descent methods on hard least squares problem

+ gradient descent with 
    + constant stepsize
    + borzilai borwein stepsize
+ accelerated gradient descent

In [ ]:
using Pkg; Pkg.activate(".")
using Revise
using PyPlot
import LinearAlgebra: norm
import SparseArrays: spdiagm, sprand, spzeros


include("Opt.jl")
import .Opt

In [ ]:
n = Int64(1e3)
f, grad! = Opt.hard_leastsquares_problem(n)
x0 = spzeros(n)
α = 0.001
n_iterations = 1000

methods = [
    "gd" => :gradient_descent,
    "gd_borzilaiborwein" => :gradient_descent_barzilaiborwein,
    "gd_nesterov" => :gradient_descent_nesterov]
n_methods = size(methods, 1)

In [ ]:
fs = zeros(n_iterations, n_methods)
grad_norm = zeros(n_iterations, n_methods)

for (i, (k, v)) in enumerate(methods)
    descent_method = getfield(Opt, v)
    
    function access_state(state)
        fs[state.k,i] = state.f
        grad_norm[state.k,i] = norm(state.g)
    end
    
    println("running $k")
    
    descent_method(x0, f, grad!;
        α = α,
        n_iterations = n_iterations,
        access_state = access_state)
end

In [ ]:
xs = 1:n_iterations

figure(figsize=(12,6))
suptitle("hard least squares problem (n=$n)")


ax = subplot(121)
ax.set_xscale("log")
ax.set_yscale("log")
ax.spines["right"].set_visible(false)
ax.spines["top"].set_visible(false)

for (i, (k,v)) in enumerate(methods)
    plot(xs, fs[:,i], label=k)
end
ylabel("f(xᵏ)")
xlabel("k")
legend(loc="lower left")


ax = subplot(122)
ax.set_xscale("log")
ax.set_yscale("log")
ax.spines["right"].set_visible(false)
ax.spines["top"].set_visible(false)

for (i, (k,v)) in enumerate(methods)
    plot(xs, grad_norm[:,i], label=k)
end
ylabel("‖∇f(xᵏ)‖")
xlabel("k")
legend(loc="lower left")


savefig("summary/assets/compare_gd_$n.png")

In [ ]:
ns = [Int64(1e5)]

for n in ns

    ############################
    ## prepare problem
    ############################
    
    f, grad! = Opt.hard_leastsquares_problem(n)
    x0 = spzeros(n)
    α = 0.001
    n_iterations = 1000

    methods = [
        "gd" => :gradient_descent,
        "gd_borzilaiborwein" => :gradient_descent_barzilaiborwein,
        "gd_nesterov" => :gradient_descent_nesterov]
    n_methods = size(methods, 1)
    
    
    ############################
    ## run algorithm
    ############################
    
    fs = zeros(n_iterations, n_methods)
    grad_norm = zeros(n_iterations, n_methods)

    for (i, (k, v)) in enumerate(methods)
        descent_method = getfield(Opt, v)

        function access_state(state)
            fs[state.k,i] = state.f
            grad_norm[state.k,i] = norm(state.g)
        end

        println("running $k")

        descent_method(x0, f, grad!;
            α = α,
            n_iterations = n_iterations,
            access_state = access_state)
    end
    
    ############################
    ## plotting
    ############################
    xs = 1:n_iterations

    figure(figsize=(12,6))
    suptitle("hard least squares problem (n=$n)")


    ax = subplot(121)
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.spines["right"].set_visible(false)
    ax.spines["top"].set_visible(false)

    for (i, (k,v)) in enumerate(methods)
        plot(xs, fs[:,i], label=k)
    end
    ylabel("f(xᵏ)")
    xlabel("k")
    legend(loc="lower left")


    ax = subplot(122)
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.spines["right"].set_visible(false)
    ax.spines["top"].set_visible(false)

    for (i, (k,v)) in enumerate(methods)
        plot(xs, grad_norm[:,i], label=k)
    end
    ylabel("‖∇f(xᵏ)‖")
    xlabel("k")
    legend(loc="lower left")


    savefig("summary/assets/compare_gd_$n.png")
end
